## 7.4 – Trees
### Binary Trees
As discussed, it's time to use our understanding of data structures in Python to build something new, a tree.

Just as a linked list is built of node objects which reference each other, we can do the same for a tree. For a *generic tree* where each parent can have any number of children, it would be easy enough to make a node class which contains the value of the node, and then a list of children (array list or linked list, depending on application). 

If the tree is going to be restricted to a certain number of children, by far the most common is the *binary* tree, where each node can have at most two. This is nice because we can just use two references in our node class:

In [1]:
class BinaryTreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
    def __str__(self):
        return str(self.value)

Hopefully this looks straightforward enough and you can see how node objects can be built recursively to create a binary tree of any depth.

Like the *head* of a linked list, we refer to the top node in a tree as the *root*.

#### Binary Search Trees
The binary tree can be used in many more advanced data structures, and we are going to focus on one of them: the binary search tree (BST). This is a binary tree where there is an *ordering* defined between a node and the value of its children on either side. For each node called `node`:
* `node.left.value < node.value`
* `node.right.value >= node.value`

Here is an example of a binary search tree where the values maintain the ordering:

<img src="./resources/binary_search_tree.svg" width=300 />

So for example take the root note: its value is 8. The value of the node on the left is 3 which is less than 8. The value on the right is 10 which is greater than 8.

To add a new node into this tree we must start at the root and then traverse the hierarchy to find the right place. Let's explain by doing: there is a Python implementation below.

In [2]:
class BinarySearchTree:
    def __init__(self):
        self.root = None

    def add(self, value):
        if self.root is None:
            self.root = BinaryTreeNode(value)
        else:
            ptr = self.root
            while True:
                if value < ptr.value:
                    if ptr.left is None:
                        ptr.left = BinaryTreeNode(value)
                        break
                    else:
                        ptr = ptr.left
                else:
                    if ptr.right is None:
                        ptr.right = BinaryTreeNode(value)
                        break
                    else:
                        ptr = ptr.right

The code to iteratively add an element to the binary search tree is a bit messy – notice for ease we are using a `while True` loop which we later use `break` to escape from. It's a pattern I try to avoid when I can but in this case it does the job.

An alternative would be to include the `.add(…)` method inside the node class, and make it recursive, which ends up looking [slightly neater](./resources/bstn.py). But I wanted to stick with the same pattern we've been using this week, where the `struct`-like syntax is used for the nodes and the main data structure class does all the wrangling.

Notice I haven't included any test code. It's actually quite a pain to check whether this is working because it's hard to inspect the entire tree structure – this would be easier in a debugger. One option is to construct a specific tree. The code below should construct the exact same tree in the image above. But this is not as simple as just using the same numbers: we must insert them into the tree in a certain order too. Hopefully this is obvious: the first value becomes the root, no matter what it is, so we must start with 8.

In [3]:
bst = BinarySearchTree()
bst.add(8)
bst.add(3)
bst.add(1)
bst.add(10)
bst.add(6)
bst.add(14)
bst.add(4)
bst.add(7)
bst.add(13)

# expect to get 7
print(bst.root.left.right.right)

7


Checking individual items based on our visual representation is *okay*, but we will have more overall confidence that the tree is working if we move onto actually *using* it.

#### Binary *Search* Trees
Why is it called a binary search tree? Because the structure makes it extremely easy to perform a binary search!

In [4]:
class BinarySearchTree:
    def __init__(self):
        self.root = None

    def add(self, value):
        if self.root is None:
            self.root = BinaryTreeNode(value)
        else:
            ptr = self.root
            while True:
                if value < ptr.value:
                    if ptr.left is None:
                        ptr.left = BinaryTreeNode(value)
                        break
                    else:
                        ptr = ptr.left
                else:
                    if ptr.right is None:
                        ptr.right = BinaryTreeNode(value)
                        break
                    else:
                        ptr = ptr.right
                        
    def contains(self, target):
        if self.root is None:
            return False
        
        ptr = self.root
        while True:
            if target == ptr.value:
                return True
        
            if target < ptr.value:
                if ptr.left is None:
                    return False
                else:
                    ptr = ptr.left
            else:
                if ptr.right is None:
                    return False
                else:
                    ptr = ptr.right
                    
                    
bst = BinarySearchTree()
bst.add(8)
bst.add(3)
bst.add(1)
bst.add(10)
bst.add(6)
bst.add(14)
bst.add(4)
bst.add(7)
bst.add(13)

# expect True
print(bst.contains(7))

# expect False
print(bst.contains(9))

True
False


So the binary search tree structure allows us to store items and ensure $O(\log(n))$ retrieval. Not as good as the hash set, but the tree structure itself uses less memory.

Hopefully you can already start to think of modifications to make this useful in various situations. The `value` of each node does not need to only be integers, it could be arbitrary objects, provided they support `==` and `<` (or we could allow the user to provide a `key` function to define their value). Now notice that we might have objects in the tree that we want to retrieve based on a single value: e.g. a user's ID number. Rather than just returning `True` and `False` (membership), we could return the object itself. Hence we have an efficient way of searching for a user by their ID number to retrieve the rest of their details, for example.

If we use a list to store the objects we would need to ensure the list was always sorted to enable binary search. Effectively, we'd need to perform a single round of insertion sort every time we added an item into the list, which would be $O(n)$. Using the binary search tree we can expect average $O(\log(n))$ performance – adding an item is the same logic as performing a binary search.

#### Tree Traversal
Suppose we want to get the entire tree's contents, one item at a time. We can actually do this in various ways, it is called *tree traversal*. 

For the binary *search* tree, there is one specific traversal order that is particularly useful, it is called *in-order* traversal, and is also known by the initialism LNR – left, node, right. This is the order you should perform for each node: first check any nodes to the left recursively, then check the node itself, then check the nodes to the right recursively.

The meaning of “check” depends on application. It's common to demonstrate this traversal by calling `print` at each stage. Suppose we applied this to the example tree from before. Remember, at *each* node think: “left, print node, right”. Starting at the root, we go left, but then we get to a new node so we go left again. The first node to print will be the 1, because it cannot go any further left.

Have a look at the animation below:

<br /><video controls loop autoplay width=400 src="./resources/in_order.mp4">
</video>

Traversing a binary search tree *in-order* results in *sorted* order of elements.

As we know, we should avoid using `print` within a data structure – maybe the caller wants to convert the tree into a list instead? Some people include a function as a parameter which is applied to each node value – this supports `print` fine but can be a bit messy for other applications. 

Thankfully, Python's generator syntax with `yield` works beautifully here. We can use `yield from` to yield recursively – so in other words, the recursive call creates a generator, and the top level generator knows to yield each element from the recursive generator:

In [5]:
class BinarySearchTree:
    def __init__(self):
        self.root = None

    def add(self, value):
        if self.root is None:
            self.root = BinaryTreeNode(value)
        else:
            ptr = self.root
            while True:
                if value < ptr.value:
                    if ptr.left is None:
                        ptr.left = BinaryTreeNode(value)
                        break
                    else:
                        ptr = ptr.left
                else:
                    if ptr.right is None:
                        ptr.right = BinaryTreeNode(value)
                        break
                    else:
                        ptr = ptr.right
                        
    def contains(self, target):
        if self.root is None:
            return False
        
        ptr = self.root
        while True:
            if target == ptr.value:
                return True
        
            if target < ptr.value:
                if ptr.left is None:
                    return False
                else:
                    ptr = ptr.left
            else:
                if ptr.right is None:
                    return False
                else:
                    ptr = ptr.right
                    
    def in_order(self):
        def traverse(node):
            if node.left is not None:
                yield from traverse(node.left)
            yield node.value
            if node.right is not None:
                yield from traverse(node.right)

        return traverse(self.root)
                    
                    
bst = BinarySearchTree()
bst.add(8)
bst.add(3)
bst.add(1)
bst.add(10)
bst.add(6)
bst.add(14)
bst.add(4)
bst.add(7)
bst.add(13)

print(list(bst.in_order()))

[1, 3, 4, 6, 7, 8, 10, 13, 14]


#### Tree Sort
If we have a collection of $n$ items to sort, then on average we'd expect each addition to the tree to be $O(\log(n))$, meaning building the full tree is $O(n \log(n))$. Traversing the tree is just $O(n)$, and since we ignore the lower growth terms in a sum (consecutive actions), building a binary search tree is actually a viable $O(n \log(n))$ sorting algorithm. It is on average a very effective way of *keeping* a sorted structure, because each addition is just $O(\log(n))$ rather than the $O(n)$ you'd expect if you tried to keep a list sorted with insertion sort.

Binary search trees are not foolproof, the construction order can make a big difference. If we construct a binary search tree from an already sorted list, we actually get a linked list as a result! So the worst case performance degrades to $O(n^2)$. There are ways of creating [self-balancing binary search trees](https://en.wikipedia.org/wiki/Self-balancing_binary_search_tree) to try to avoid this.

#### Traversal
In-order traversal for sorting is the most useful method for traversing a binary search tree. But for other binary trees, pre-order (NLR) or post-order (LRN) can be useful. You can read more online, [such as this article](https://en.wikipedia.org/wiki/Tree_traversal), and you will see another application in this week's exercise sheet.

## What Next?
Trees and graphs are useful in lots of situations, including AI. For example: if you are trying to write an AI that plays a game like [noughts and crosses](https://en.wikipedia.org/wiki/Tic-tac-toe), you can search an implicit tree containing all of the possible moves to [work out which one is optimal](https://en.wikipedia.org/wiki/Minimax). Python doesn't have any built in structures to build trees for you, but now you can see how easy it is to build them yourself.

Once you are done with this material, head back to Engage to move onto the wrap up and the aforementioned exercise sheet.